In [1]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, ParameterGrid
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
import warnings
warnings.filterwarnings("ignore")


In [2]:
# training dataset loading
Training_csv_file_path = './data/TrainingSet/mRNA_sublocation_TrainingSet_NC-BERTdata.csv'
Training_data= pd.read_csv(Training_csv_file_path)
# separate sequence features and labels
X_Train = Training_data.drop(columns=['label']).values
y_Train = Training_data['label'].values
# the training data and test data are standardized
sc = StandardScaler()
sc.fit(X_Train)
X_Train = sc.transform(X_Train)

In [3]:
from sklearn.decomposition import PCA
import numpy as np
# Initialize PCA and set the number of principal components
pca = PCA(n_components=72)
# Fit PCA on the training data and transform the training data to its principal components
X_Train = pca.fit_transform(X_Train)


In [4]:
# Define hyperparameter ranges
param_grid = {
    'learning_rate':[0.08,0.1],
    'depth':[5, 6],
    'rsm':[0.9, 1],
    'subsample':[0.6, 1.0],
    'min_data_in_leaf':[140,150]
}

# Create all combinations of hyperparameters
grid = list(ParameterGrid(param_grid))

# Initialize the result list
results = []

# 100 times 5-fold cross-validation
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=100, random_state=42)
rskf.get_n_splits(X_Train, y_Train)

# Open the CSV file for writing
with open('./result/CatBoost-100times-5-fold cv.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['learning_rate', 'depth','rsm','subsample','min_data_in_leaf', 'accuracy', 'precision', 'recall', 'f1'])
    
    # Perform a grid search
    for params in tqdm(grid, desc="Hyperparameter search"):
        acc_scores = []
        prec_scores = []
        rec_scores = []
        f1_scores = []
        
        for i, (train_index, test_index) in enumerate(rskf.split(X_Train, y_Train)):
            X_train, X_test = X_Train[train_index], X_Train[test_index]
            y_train, y_test = y_Train[train_index], y_Train[test_index]
            clf = CatBoostClassifier(iterations=1000,
                                     learning_rate=params['learning_rate'],
                                     depth=params['depth'],
                                     rsm=params['rsm'],
                                     subsample=params['subsample'],
                                     min_data_in_leaf=params['min_data_in_leaf'],
                                     logging_level='Silent',
                                     early_stopping_rounds=50)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            acc_scores.append(accuracy_score(y_test, y_pred))
            prec_scores.append(precision_score(y_test, y_pred))
            rec_scores.append(recall_score(y_test, y_pred))
            f1_scores.append(f1_score(y_test, y_pred))
        
        # Save scores for this hyperparameter combination
        for acc, prec, rec, f1 in zip(acc_scores, prec_scores, rec_scores, f1_scores):
            writer.writerow([params['learning_rate'], params['depth'],params['rsm'],params['subsample'],params['min_data_in_leaf'], acc, prec, rec, f1])

        acc_mean, acc_std = np.mean(acc_scores), np.std(acc_scores)
        prec_mean, prec_std = np.mean(prec_scores), np.std(prec_scores)
        rec_mean, rec_std = np.mean(rec_scores), np.std(rec_scores)
        f1_mean, f1_std = np.mean(f1_scores), np.std(f1_scores)
        print(f" params: {params}")
        print(f" acc_mean: {acc_mean}, acc_std: {acc_std}")
        print(f" prec_mean: {prec_mean}, prec_std: {prec_std}")
        print(f" rec_mean: {rec_mean}, rec_std: {rec_std}")
        print(f" f1_mean: {f1_mean}, f1_std: {f1_std}")

Hyperparameter search:   3%|█▋                                                     | 1/32 [54:51<28:20:31, 3291.35s/it]

 params: {'depth': 5, 'learning_rate': 0.08, 'min_data_in_leaf': 140, 'rsm': 0.9, 'subsample': 0.6}
 acc_mean: 0.6893538196504883, acc_std: 0.012872690689444868
 prec_mean: 0.6653079843264181, prec_std: 0.018484217250265302
 rec_mean: 0.5760327069269598, rec_std: 0.023030538749052178
 f1_mean: 0.6172004237506367, f1_std: 0.017419098465156428


Hyperparameter search:   6%|███▎                                                 | 2/32 [1:49:16<27:17:57, 3275.92s/it]

 params: {'depth': 5, 'learning_rate': 0.08, 'min_data_in_leaf': 140, 'rsm': 0.9, 'subsample': 1.0}
 acc_mean: 0.68674112075588, acc_std: 0.01389146812078485
 prec_mean: 0.6617377047231503, prec_std: 0.02029833151076825
 rec_mean: 0.5735337225567111, rec_std: 0.023090832258132406
 f1_mean: 0.6142118110342081, f1_std: 0.01799072221372561


Hyperparameter search:   9%|████▉                                                | 3/32 [2:30:01<23:19:59, 2896.52s/it]

...

 params: {'depth': 6, 'learning_rate': 0.1, 'min_data_in_leaf': 150, 'rsm': 0.9, 'subsample': 1.0}
 acc_mean: 0.6878567544815974, acc_std: 0.01332297138133157
 prec_mean: 0.6648492103067123, prec_std: 0.019687736824896915
 rec_mean: 0.5704328650489571, rec_std: 0.022123501027825253
 f1_mean: 0.613780136599175, f1_std: 0.017409246720770006


Hyperparameter search:  97%|██████████████████████████████████████████████████▍ | 31/32 [40:51:25<1:21:57, 4917.35s/it]

 params: {'depth': 6, 'learning_rate': 0.1, 'min_data_in_leaf': 150, 'rsm': 1, 'subsample': 0.6}
 acc_mean: 0.6901177359480011, acc_std: 0.013142871948553288
 prec_mean: 0.6681291647458178, prec_std: 0.01928368548379546
 rec_mean: 0.5723257191510065, rec_std: 0.022250082085278945
 f1_mean: 0.616277904614149, f1_std: 0.01740615680982774


Hyperparameter search: 100%|██████████████████████████████████████████████████████| 32/32 [41:31:30<00:00, 4671.58s/it]

 params: {'depth': 6, 'learning_rate': 0.1, 'min_data_in_leaf': 150, 'rsm': 1, 'subsample': 1.0}
 acc_mean: 0.6878738390626941, acc_std: 0.013137657810906388
 prec_mean: 0.665074904196652, prec_std: 0.01931631231038253
 rec_mean: 0.5698833911086785, rec_std: 0.022364000032063652
 f1_mean: 0.6135549255926471, f1_std: 0.01738384070420942
